# Load Iris Dataset

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import joblib
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data
y = iris.target

def display_scores(scores):
    print("Scores:", scores)
    print("Means:", scores.mean())
    print("Standard Deviation:", scores.std())

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

rf_clf = RandomForestClassifier()
rf_clf.fit(X_train, y_train)
rf_scores = cross_val_score(rf_clf, X_train, y_train, scoring = 'accuracy', cv = 3)
display_scores(rf_scores)
joblib.dump(rf_clf, 'models/rf_clf.joblib')

Scores: [0.975 0.9   0.975]
Means: 0.9500000000000001
Standard Deviation: 0.035355339059327355


['models/rf_clf.joblib']

In [23]:
%%writefile scripts/flask_api.py

import joblib
import numpy as np
from flask import Flask, request
from flask import Response
import json

app = Flask(__name__)

@app.route('/')
def home_endpoint():
    return 'Hello World!'

@app.route('/predict', methods=['POST'])
def predict_endpoint():
    test_data = request.get_json()
    test_data = np.array(test_data)
    predictions = model.predict(test_data)
    response = json.dumps(predictions.tolist())
    return Response(response, status=200, mimetype="application/json")

if __name__ == '__main__':
    model = joblib.load('rf_clf.joblib')
    app.run(host='0.0.0.0', port=8080)

Overwriting scripts/flask_api.py


In [17]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import numpy as np
import requests
import json

iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

url = 'http://localhost:8080/predict'
headers = {
    'Content-type': "application/json"
}

data = json.dumps(X_test.tolist())
response = requests.post(url, headers=headers, data=data)
predictions = np.array(json.loads(response.text))
print(f'Predictions: {predictions}')
print(f'Accuracy Score: {accuracy_score(predictions, y_test)}')

Overwriting scripts/predict.py


In [21]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import numpy as np
import requests
import json

iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

url = 'http://flask-api-dev.eu-west-2.elasticbeanstalk.com/predict'
headers = {
    'Content-type': "application/json"
}

data = json.dumps(X_test.tolist())
response = requests.post(url, headers=headers, data=data)
predictions = np.array(json.loads(response.text))
print(f'Predictions: {predictions}')
print(f'Accuracy Score: {accuracy_score(predictions, y_test)}')

Predictions: [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
Accuracy Score: 1.0


In [20]:
!python scripts/eb_predict.py

Predictions: [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
Accuracy Score: 1.0


Follow the prompts that appear on the screen. For this project, the default values for all the options will suffice. 
Elastic Beanstalk will automatically handle all the infrastructure deployment required to help you scale your project so you can concentrate on developing the codebase. It will go through the process of automatically creating all the necessary resources such as launching an EC2 instance, setting load balancer and auto scaling groups.

Upon successful deployment, Elastic Beanstalk will return an address where the application will be available. In this example, the application is available at http://flask-api-dev.eu-west-2.elasticbeanstalk.com.
6. Serving our model as an API
Now our application is deployed in the cloud, we can access it like we did previously when testing our application locally. The only major difference this time is the address where we will be accessing the API. 
Replace the URL from http://localhost:8080/predict to the URL of your application. In this case, I will be replacing it with http://flask-api-dev.eu-west-2.elasticbeanstalk.com.